# Main

In [71]:
import os
import json
import time
import hashlib
import requests
import pandas as pd
import pandas_gbq
from datetime import timedelta
from google.cloud import storage, bigquery

from datetime import datetime

## Auth

In [3]:
path = 'credentials/service_account.json'
client = storage.Client.from_service_account_json(path)

bucket_name = "valorant_data"

bucket = client.bucket(bucket_name)

client_gbq = bigquery.Client.from_service_account_json(path)
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = os.path.join(path)

## Get all Stored Matchs

In [4]:
folder_name = "match"

folder = bucket.blob(folder_name)

stored_matchs = []

# Lista todos os arquivos dentro da pasta
for blob in bucket.list_blobs():#prefix=folder_name):
    stored_matchs.append(blob.name.replace('match/', '').replace('.json', ''))

## Declare execution time

In [6]:
date_time = datetime.now()
                    
if time.tzname[0] == 'UTC':
    date_time = date_time - timedelta(hours=3)

year = date_time.strftime('%Y')
year_month = date_time.strftime('%Y%m')
day = date_time.strftime('%d')
timestamp = int(date_time.timestamp())
dt_extract = date_time.strftime('%Y-%m-%dT%H:%m:%S')

date_time = date_time.strftime('%Y%m%d_%H%m%S')

## Target Users

In [7]:
users = {
    "viikset": "BR1",
    "TADALA": "TADAL",
    "DIGAOTJS": "BR1",
    "mBirth": "BR1",
    "Kubata": "4881"
}

puuids = {
    "viikset": "f2f7c867-27ba-5291-a23c-bbdcfc079e76",
    "TADALA": "35a2368e-586f-53ec-8ee8-ea9ebdca7d6f",
    "DIGAOTJS": "2c51a2e4-0dcf-53f5-ae85-cd4d7d79f663",
    "mBirth": "6c241ab0-1c2f-5d5b-8908-4e809df8f6ca",
    "Kubata": "b2310858-1741-5d6f-8499-cb1f85bd182a"
}

## Layer Input

In [12]:
session_matchs = []

for name, tag in users.items():
    
    # Match History
    endpoint = 'matches'
    response = requests.get(f"https://api.henrikdev.xyz/valorant/v3/{endpoint}/na/{name}/{tag}?filter=competitive")
    
    if response.status_code != 200:
        
        error_id = hashlib.md5((dt_extract+name+tag).encode('utf-8')).hexdigest()
        df_error = pd.DataFrame([{'id': error_id, 'datatime': dt_extract, 'name': name, 'tag': tag, 'endpoint': endpoint, 'status': response.status_code}])
        
        df_error['datetime'] = pd.to_datetime(df_error['datetime'])

        schema = [
            {'name': 'id', 'type': 'STRING'},
            {'name': 'datetime', 'type': 'DATETIME'},
            {'name': 'name', 'type': 'STRING'},
            {'name': 'tag', 'type': 'STRING'},
            {'name': 'endpoint', 'type': 'STRING'},
            {'name': 'status', 'type': 'INTEGER'}
        ]
        
        df_error.to_gbq(
            f'logs_valorant.api_error_logs',
            'thales-1615931464192',
            if_exists='append',
            table_schema=schema
        )   
        
    else:

        for match in response.json()['data']:

            matchid = match['metadata']['matchid']

            if matchid not in stored_matchs and matchid not in session_matchs:
                
                session_matchs.append(matchid)
                session_matchs = list(set(session_matchs))

                # Save file
                file_name = f'match/{matchid}.json'

                with open(f'{file_name}', 'w') as f:
                    json.dump(match, f)

                blob = bucket.blob(f"{file_name}")
                blob.upload_from_filename(file_name)

status code: 200
Match 77176f97-6d0c-4ba2-b022-2a20cabc7975 do not exist in storage...
Match 30020147-5ab6-4ac9-86d6-497cf873f15b do not exist in storage...
Match b25ec6b0-36d1-4918-b283-503a290c2983 do not exist in storage...
Match 48a1e6b4-d551-4ca1-98a6-e85081ca0fd1 do not exist in storage...
Match 3988d557-de2a-498e-8db2-e4484146a3ba do not exist in storage...
status code: 200
status code: 200
status code: 200
Match fa668a51-832d-403b-a94a-13c0d9eaa922 do not exist in storage...
Match 9beb3bea-c341-4341-a3d5-d89f5f19f903 do not exist in storage...
Match 83ea191d-3670-4bc0-b2ea-83bfd3cf5183 do not exist in storage...
Match 188849fd-7f4d-4b21-9ad4-5b00c4e92537 do not exist in storage...
Match 0d9063f2-4b7f-4737-8244-4c5e32924dfa do not exist in storage...
status code: 200
Match 3d1e95f9-7c31-42ce-9f1f-ac5b772551e8 do not exist in storage...
Match 23106ddd-5db8-459c-9169-5ba80979ea8a do not exist in storage...
Match 88e8a918-2f57-453c-999b-c18e9a8f8ed3 do not exist in storage...
Match

In [84]:
folder = bucket.blob(folder_name)

df_matches = pd.DataFrame()

# Lista todos os arquivos dentro da pasta
for blob in bucket.list_blobs():#prefix=folder_name):
    #matchs_post_run.append(blob.name.replace('match/', '').replace('.json', ''))
    df_matches = pd.concat([df_matches, pd.DataFrame([blob.name.replace('match/', '').replace('.json', '')])])

df_matches.columns = ['matchid']

df_matches['datetime'] = dt_extract

df_matches = df_matches[['datetime', 'matchid']]
df_matches.reset_index(inplace=True, drop=True)

df_matches['datetime'] = pd.to_datetime(df_matches['datetime'])

schema = [
    {'name': 'datetime', 'type': 'DATETIME'},
    {'name': 'matchid', 'type': 'STRING'}
]

df_matches.to_gbq(
    f'logs_valorant.match_in_storage',
    'thales-1615931464192',
    if_exists='append',
    table_schema=schema
)   

## Get Account Data

In [ ]:
all_match_ids = []


for name, tag in users.items():
    
    # Account Data
    endpoint = 'account'
    response = requests.get(f"https://api.henrikdev.xyz/valorant/v1/{endpoint}/{name}/{tag}")
    
    if response.status_code != 200:
        # Add Alert
        print('error')
    else:
        pass
    
   
    with open(f'account/{date_time}_{name}_{tag}.json', 'w') as f:
        json.dump(response.json(), f)
    
    
        
    # MMR Data
    endpoint = 'mmr'
    response = requests.get(f"https://api.henrikdev.xyz/valorant/v1/{endpoint}/na/{name}/{tag}")
    
    if response.status_code != 200:
        # Add Alert
        print('error')
    else:
        pass
    
    with open(f'mmr_data/{date_time}_{name}_{tag}.json', 'w') as f:
        json.dump(response.json(), f)
        
        
    # MMR History
    endpoint = 'mmr-history'
    response = requests.get(f"https://api.henrikdev.xyz/valorant/v1/{endpoint}/na/{name}/{tag}")
    
    if response.status_code != 200:
        # Add Alert
        print('error')
    else:
        pass
    
    with open(f'mmr_history/{date_time}_{name}_{tag}.json', 'w') as f:
        json.dump(response.json(), f)


    # Match History
    endpoint = 'matches'
    response = requests.get(f"https://api.henrikdev.xyz/valorant/v3/{endpoint}/na/{name}/{tag}?filter=competitive")
    
    if response.status_code != 200:
        # Add Alert
        print('error')
    else:
        pass
    
    with open(f'match_history/{date_time}_{name}_{tag}.json', 'w') as f:
        json.dump(response.json(), f)
        
        
    # Listing All Match IDs
    metadata = []
    for i in response.json()['data']:
        all_match_ids.append(i['metadata']['matchid'])


In [ ]:
all_match_ids = list(set(all_match_ids))

In [ ]:
all_match_ids = list(set(all_match_ids) - set(stored_matchs))

In [ ]:
for matchid in all_match_ids:
    
    endpoint = 'match'
    response = requests.get(f"https://api.henrikdev.xyz/valorant/v2/{endpoint}/{matchid}")
    
    if response.status_code != 200:
        # Add Alert
        print('error')
    else:
        pass
    
    # Save file
    file_name = f'{endpoint}/{matchid}.json'
    
    with open(f'{file_name}', 'w') as f:
        json.dump(response.json(), f)
        
    blob = bucket.blob(f"{file_name}")
    blob.upload_from_filename(file_name)
    

In [ ]:
endpoint = 'match'
response = requests.get(f"https://api.henrikdev.xyz/valorant/v2/{endpoint}/{matchId}")

In [ ]:
json_data = response.json()

In [ ]:
pd.json_normalize(json_data['metadata'])

In [ ]:
pd.DataFrame([json_data['players']['all_players'][0]])

In [ ]:
all_match_ids = []

df_account = pd.DataFrame()
df_mmr_data = pd.DataFrame()
df_mmr_history = pd.DataFrame()
df_matchs = pd.DataFrame()


for name, tag in users.items():
    
    print(f'Get data for {name}...')
    
    # Accounts Data
    #print(f'Get account data for {name}...')
    response = requests.get(f"https://api.henrikdev.xyz/valorant/v1/account/{name}/{tag}")
    
    with open(f'account/{timestamp}_{name}_{tag}.json', 'w') as f:
        json.dump(response.json(), f)
        
    df_account = pd.concat([df_account, pd.json_normalize(response.json()['data'])])
        
        
    # MMR Data
    #print(f'Get MMR data for {name}...')
    response = requests.get(f"https://api.henrikdev.xyz/valorant/v1/mmr/na/{name}/{tag}")
    
    with open(f'mmr_data/{timestamp}_{name}_{tag}.json', 'w') as f:
        json.dump(response.json(), f)
        
    df_mmr_data = pd.concat([df_mmr_data, pd.json_normalize(response.json()['data'])])  
        
    # MMR History
    #print(f'Get MMR History data for {name}...')
    response = requests.get(f"https://api.henrikdev.xyz/valorant/v1/mmr-history/na/{name}/{tag}")
    
    with open(f'mmr_history/{timestamp}_{name}_{tag}.json', 'w') as f:
        json.dump(response.json(), f)
        
        
    temp_mmr_history = pd.json_normalize(response.json()['data'])
    temp_mmr_history['name'] = response.json()['name']
    temp_mmr_history['tag'] = response.json()['tag']
    
    df_mmr_history = pd.concat([df_mmr_history, temp_mmr_history])
    
    # Match History
    #print(f'Get Match History data for {name}...')
    response = requests.get(f"https://api.henrikdev.xyz/valorant/v3/matches/na/{name}/{tag}?filter=competitive")
    
    with open(f'match_history/{timestamp}_{name}_{tag}.json', 'w') as f:
        json.dump(response.json(), f)
        
    # Listing All Match IDs
    metadata = []
    for i in response.json()['data']:
        all_match_ids.append(i['metadata']['matchid'])
        
        metadata.append(i['metadata'])
        
    df_matchs = pd.concat([df_matchs, pd.DataFrame(metadata)])
    
    print('\n')

df_account['datetime'] = datetime
df_mmr_data['datetime'] = datetime

# Get All Match History

In [ ]:
all_match_ids = list(set(all_match_ids))

In [ ]:
metadata = []
rounds = []
kills = []

# Players Dataframes
session_playtime = pd.DataFrame()
behavior = pd.DataFrame()
platform = pd.DataFrame()
ability_casts = pd.DataFrame()
assets = pd.DataFrame()
stats = pd.DataFrame()
economy = pd.DataFrame()
player = pd.DataFrame()


df_players = pd.DataFrame()
df_teams = pd.DataFrame()

for matchid in all_match_ids:
    
    # Request
    response = requests.get(f"https://api.henrikdev.xyz/valorant/v2/match/{matchid}")
    
    # Save file
    with open(f'match/{matchid}.json', 'w') as f:
        json.dump(response.json(), f)
        
    #match = response.json()['data']
    #    
    ## Match Metadata    
    #metadata.append(match['metadata'])
    #
    ## Match Players
    #json_players = match['players']['all_players'].copy()
#
    #for player in json_players:
#
    #    puuid = player['puuid']
#
    #    temp_session_playtime = player.pop('session_playtime')
    #    temp_session_playtime = pd.json_normalize(temp_session_playtime)
    #    temp_session_playtime['matchid'] = matchid
    #    temp_session_playtime['puuid'] = puuid
    #    session_playtime = pd.concat([session_playtime, temp_session_playtime])
#
    #    temp_behavior = player.pop('behavior') 
    #    temp_behavior = pd.json_normalize(temp_behavior)
    #    temp_behavior['matchid'] = matchid
    #    temp_behavior['puuid'] = puuid
    #    behavior = pd.concat([behavior, temp_behavior])
#
    #    temp_platform = player.pop('platform')
    #    temp_platform = pd.json_normalize(temp_platform)
    #    temp_platform['matchid'] = matchid
    #    temp_platform['puuid'] = puuid
    #    platform = pd.concat([platform, temp_platform])
#
    #    temp_ability_casts = player.pop('ability_casts')
    #    temp_ability_casts = pd.json_normalize(temp_ability_casts)
    #    temp_ability_casts['matchid'] = matchid
    #    temp_ability_casts['puuid'] = puuid
    #    ability_casts = pd.concat([ability_casts, temp_ability_casts])
    #    
    #    temp_assets = player.pop('assets')
    #    temp_assets = pd.json_normalize(temp_assets)
    #    temp_assets['matchid'] = matchid
    #    temp_assets['puuid'] = puuid
    #    assets = pd.concat([assets, temp_assets])
#
    #    temp_stats = player.pop('stats')
    #    temp_stats = pd.json_normalize(statstemp_stats
    #    temp_stats['matchid'] = matchid
    #    temp_stats['puuid'] = puuid
    #    stats = pd.concat([stats, temp_stats])
#
    #    temp_economy = player.pop('economy')
    #    temp_economy = pd.json_normalize(temp_economy)
    #    temp_economy['matchid'] = matchid
    #    temp_economy['puuid'] = puuid
    #    economy = pd.concat([economy, temp_economy])
    #                                   
    #    temp_player = pd.json_normalize(pltemp_playerayer)
    #    temp_player['matchid'] = matchid
    #    player = pd.concat([player, temp_player])
    #
    ## Match Teams
    #temp_teams = pd.json_normalize(match['teams'])
    #temp_teams['matchid'] = matchid
    #df_teams = pd.concat([df_teams, temp_teams])
    #
    #
    #rounds.append(match['rounds'])
    #
    #
    #kills.append(match['kills'])
    #    
    ##break
    #
#df_metadata = pd.DataFrame(metadata)


In [ ]:
match = response.json()['data']

json_rounds = match['rounds'].copy()

plant_events = pd.DataFrame()

count = 1

for row_round in json_rounds:
    
    # Plant Events
    plant_events = row_round.pop('plant_events')
    player_locations_on_plant = plant_events.pop('player_locations_on_plant')
    
    plant_events = pd.json_normalize(plant_events)
    plant_events['matchid'] = matchid
    plant_events['round'] = count
    
    player_locations_on_plant = pd.json_normalize(player_locations_on_plant)
    player_locations_on_plant['matchid'] = matchid
    player_locations_on_plant['round'] = count
    
    # Defuse Events
    defuse_events = row_round.pop('defuse_events')
    player_locations_on_defuse = defuse_events.pop('player_locations_on_defuse')
    
    defuse_events = pd.json_normalize(defuse_events)
    defuse_events['matchid'] = matchid
    defuse_events['round'] = count

    player_locations_on_defuse = pd.json_normalize(player_locations_on_defuse)
    player_locations_on_defuse['matchid'] = matchid
    player_locations_on_defuse['round'] = count
    
    # Player Stats
    player_stats = row_round.pop('player_stats')
    
    for plyr in player_stats:
        
    
    count += 1
    break
    
    

In [ ]:
for plyr in player_stats:
        break

In [ ]:
plyr.keys()

In [ ]:
plyr['stayed_in_spawn']

In [ ]:
pd.json_normalize(plyr['damage_events'])